In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import joblib
file_path = '/content/drive/MyDrive/reddit/balanced_comment/no_include_sample_comm2.csv'
columns_to_read = ['id', 'subreddit', 'total_text_no_punc', 'total_text', 'year']


# Read only the first 100 rows from the CSV file
comments = pd.read_csv(file_path, usecols=columns_to_read)
comments

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
nb_vectorizer_bin = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/nb_tfidf_vectorizer_bin.joblib')
nb_classifier_bin = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/nb_naive_bayes_classifier_bin.joblib')



In [ ]:
#new_texts = ["I sure do love cocaine", "The hottest day of the summer so far was drawing to a close and a drowsy silence lay over the large, square houses of Privet Drive."]

new_texts_vec = nb_vectorizer_bin.transform(comments['total_text'])
new_predictions = nb_classifier_bin.predict(new_texts_vec)
comments['nb_binary'] = new_predictions
comments['nb_binary'].mean()

In [ ]:
multi_nb = comments[comments['nb_binary'] == 1][['id','total_text']]


In [ ]:

# Load the vectorizer, multilabel binarizer, and multioutput classifier from files
nb_vect_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/nb_tfidf_vectorizer_multi.joblib')
nb_mlb_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/nb_multilabel_binarizer_multi.joblib')
nb_classifier_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/nb_multinomial_nb_multi.joblib')

# Now you can use the loaded_vectorizer_multi, loaded_mlb, and loaded_classifier_multi to make predictions on new data



In [ ]:

#new_texts_multi = ["Some new text", "Another piece of text"]
new_texts_vec_multi = nb_vect_multi .transform(multi_nb['total_text'])
new_labels_multi = nb_classifier_multi.predict(new_texts_vec_multi)
multi_nb['nb_multi'] = nb_mlb_multi.inverse_transform(new_labels_multi)

comments = pd.merge(comments,multi_nb[['id','nb_multi']], on = 'id', how = 'left')
del multi_nb


In [ ]:
import joblib

xgb_vectorizer_bin = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/xgb_tfidf_vectorizer_bin.joblib')
xgb_model_bin = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/xgboost_model_bin.joblib')


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:35:59] WARNING: /workspace/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:35:59] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


In [ ]:
#new_texts = ['blue blue blue hey', "harry potter is a meh book", "meth is good but cocaine is better", "What area in Germany?	"]

new_texts_vec = xgb_vectorizer_bin.transform(comments['total_text'])
new_texts_vec_gpu = new_texts_vec.toarray()
#new_texts_vec_gpu = cp.asarray(new_texts_vec.toarray())
new_predictions = xgb_model_bin.predict(new_texts_vec_gpu)
comments['xg_boost_bin'] = new_predictions
multi_xgb = comments[comments['xg_boost_bin'] == 1][['id','total_text']]



In [ ]:
xgb_mlb_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/xgb_multilabel_binarizer_multi.joblib')
xbg_vec_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/xgb_tfidf_vectorizer_multi.joblib')
xgb_model_multi = joblib.load('/content/drive/MyDrive/reddit/balanced_comment/xgboost_model_multi.joblib')




/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:36:05] WARNING: /workspace/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:36:05] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


In [ ]:
#new_texts = ['blue blue blue hey', "harry potter is a meh book", "meth is good but cocaine is better", "What area in Germany?	"]


new_texts_vec = xbg_vec_multi.transform(multi_xgb['total_text'])
new_texts_vec_gpu = new_texts_vec.toarray()
#new_texts_vec_gpu = cp.asarray(new_texts_vec.toarray())
multi_xgb['xg_multi'] = xgb_mlb_multi.inverse_transform(xgb_model_multi.predict(new_texts_vec_gpu))
comments = pd.merge(comments,multi_xgb[['id','xg_multi']], how = 'left', on = 'id')
comments

,id,subreddit,total_text_no_punc,total_text,year,nb_binary,nb_multi,xg_boost_bin,xg_multi
0,f973fwq,de,Kommt wahrscheinlich auf die Region an aber h...,"Kommt wahrscheinlich auf die Region an, aber h...",2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
1,f973gcz,portugal,https://www ctt pt/correio-e-encomendas/recebe...,https://www.ctt.pt/correio-e-encomendas/recebe...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
2,f973gda,portugal,O que há a conversar A premissa que te levou ...,O que há a conversar? A premissa que te levou ...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
3,f973gel,CasualUK,I’m the same but with the dark blue jeans the...,"I’m the same but with the dark blue jeans, the...",2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
4,f973giy,de,What area in Germany,What area in Germany?,2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
...,...,...,...,...,...,...,...,...,...
95,f974014,de,&gt; Ich bin selbst in der Lage zu entscheiden...,&gt; Ich bin selbst in der Lage zu entscheiden...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
96,f97403v,de,Ja da hast du absolut Recht Aber wo ist das d...,Ja da hast du absolut Recht. Aber wo ist das d...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
97,f9740ew,italy,Una sola pagina informazioni personali ben vi...,"Una sola pagina, informazioni personali ben vi...",2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count..."
98,f9740uy,unitedkingdom,Where I lived went from Lib safe a Tory-Lab ma...,Where I lived went from Lib safe a Tory-Lab ma...,2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count..."


In [8]:
import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader, Dataset

import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, recall_score
from torch.cuda.amp import autocast, GradScaler
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits




import torch
from transformers import BertTokenizer

# Assuming you have saved the model state_dict as 'bert_classifier.pth'
saved_model_path = '/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_bin.pth'

# Assuming BERTClassifier is defined in your_module
# Make sure to replace 'your_module' with the actual module where your BERTClassifier is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
# Initialize the model architecture
loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=2).to(device)  # Make sure to provide the correct configuration
#loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=2)  # Make sure to provide the correct configuration

# Load the saved state_dict
loaded_model.load_state_dict(torch.load(saved_model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# Set the model to evaluation mode
loaded_model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text for prediction
example_text = "This is an example text for sentiment analysis."

# Tokenize example_text
with torch.no_grad():
    encoding = tokenizer(example_text, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)

# Check GPU availability


# Optimized parameters
batch_size = 3000  # Adjust based on your GPU memory
prediction_list = []
length = 0

# Process data in batches
with torch.no_grad():
    for i in range(0, len(comments['total_text']), batch_size):
        batch_texts = comments['total_text'].iloc[i:i + batch_size].tolist()

        # Tokenize and move data to GPU
        encodings = tokenizer(batch_texts, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']

        # Model inference on GPU
        output = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(output, dim=1)

        # Move predictions back to CPU
        predicted_class = predicted_class.cpu().tolist()

        # Extend prediction_list
        prediction_list.extend(predicted_class)

        length = length + batch_size
        # Print progress
        if (length) % 1000 == 0:
            print(f"Processed {length} instances")

print("Processing complete.")
comments['NN_bin'] = prediction_list


Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Processed 2000 instances
Processed 4000 instances
Processed 6000 instances


KeyboardInterrupt: 

In [ ]:
multi_nn = comments[comments['NN_bin'] == 1][['id','total_text']]


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import accuracy_score, classification_report, recall_score, precision_score, f1_score
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.float32)  # Change to float for multi-label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits


import torch
from transformers import BertTokenizer

# Assuming you have saved the model state_dict as 'bert_classifier.pth'
saved_model_path = '/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_multi.pth'

# Assuming BERTClassifier is defined in your_module
# Make sure to replace 'your_module' with the actual module where your BERTClassifier is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
# Initialize the model architecture
loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=8).to(device)  # Make sure to provide the correct configuration
#loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=2)  # Make sure to provide the correct configuration

# Load the saved state_dict
loaded_model.load_state_dict(torch.load(saved_model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# Set the model to evaluation mode
loaded_model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text for prediction
example_text = "This is an example text for sentiment analysis."

# Tokenize example_text
with torch.no_grad():
    encoding = tokenizer(example_text, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)

# Check GPU availability


# Optimized parameters
batch_size = 2000  # Adjust based on your GPU memory
prediction_list = []
length = 0

# Process data in batches
with torch.no_grad():
    for i in range(0, len(multi_nn['total_text']), batch_size):
        batch_texts = multi_nn['total_text'].iloc[i:i + batch_size].tolist()

        # Tokenize and move data to GPU
        encodings = tokenizer(batch_texts, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']

        # Model inference on GPU
        output = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(output, dim=1)

        # Move predictions back to CPU
        predicted_class = predicted_class.cpu().tolist()

        # Extend prediction_list
        prediction_list.extend(predicted_class)

        length = length + batch_size
        # Print progress
        if (length) % 1000 == 0:
            print(f"Processed {length} instances")

print("Processing complete.")

class_list = ['count_fentanyl', 'count_heroin', 'count_oxy',  'count_MDMA',
              'count_INHALANTS', 'count_GHB', 'count_COCAINE', 'count_marijuana']
class_list = sorted(class_list)

multi_nn['NN_multi2'] = prediction_list
multi_nn['NN_multi'] = multi_nn.NN_multi2.map(pd.Series(class_list))



In [ ]:
multi_nn['NN_multi'] = multi_nn.NN_multi.map(pd.Series(class_list))

comments = pd.merge(comments,multi_nn[['id','NN_multi']], how = 'left', on = 'id')
del multi_nn


In [ ]:
comments

,id,subreddit,total_text_no_punc,total_text,year,nb_binary,nb_multi,xg_boost_bin,xg_multi,NN_bin,NN_multi
0,f973fwq,de,Kommt wahrscheinlich auf die Region an aber h...,"Kommt wahrscheinlich auf die Region an, aber h...",2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
1,f973gcz,portugal,https://www ctt pt/correio-e-encomendas/recebe...,https://www.ctt.pt/correio-e-encomendas/recebe...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
2,f973gda,portugal,O que há a conversar A premissa que te levou ...,O que há a conversar? A premissa que te levou ...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
3,f973gel,CasualUK,I’m the same but with the dark blue jeans the...,"I’m the same but with the dark blue jeans, the...",2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
4,f973giy,de,What area in Germany,What area in Germany?,2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,f974014,de,&gt; Ich bin selbst in der Lage zu entscheiden...,&gt; Ich bin selbst in der Lage zu entscheiden...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
96,f97403v,de,Ja da hast du absolut Recht Aber wo ist das d...,Ja da hast du absolut Recht. Aber wo ist das d...,2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
97,f9740ew,italy,Una sola pagina informazioni personali ben vi...,"Una sola pagina, informazioni personali ben vi...",2019,0,NaN,1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
98,f9740uy,unitedkingdom,Where I lived went from Lib safe a Tory-Lab ma...,Where I lived went from Lib safe a Tory-Lab ma...,2019,1,"(count_marijuana,)",1,"(count_GHB, count_INHALANTS, count_MDMA, count...",0,NaN
